In [3]:
import pandas as pd
data= pd.read_csv('global-data-on-sustainable-energy.csv')
missing_values = data.isnull().sum()
length=len(data)
missing_percentage = (missing_values / len(data)) * 100
print(missing_values)

Entity                                                                 0
Year                                                                   0
Access to electricity (% of population)                               10
Access to clean fuels for cooking                                    169
Renewable-electricity-generating-capacity-per-capita                 931
Financial flows to developing countries (US $)                      2089
Renewable energy share in the total final energy consumption (%)     194
Electricity from fossil fuels (TWh)                                   21
Electricity from nuclear (TWh)                                       126
Electricity from renewables (TWh)                                     21
Low-carbon electricity (% electricity)                                42
Primary energy consumption per capita (kWh/person)                     0
Energy intensity level of primary energy (MJ/$2017 PPP GDP)          207
Value_co2_emissions_kt_by_country                  

In [4]:
missing_percentage

Entity                                                               0.000000
Year                                                                 0.000000
Access to electricity (% of population)                              0.274048
Access to clean fuels for cooking                                    4.631406
Renewable-electricity-generating-capacity-per-capita                25.513839
Financial flows to developing countries (US $)                      57.248561
Renewable energy share in the total final energy consumption (%)     5.316525
Electricity from fossil fuels (TWh)                                  0.575500
Electricity from nuclear (TWh)                                       3.453001
Electricity from renewables (TWh)                                    0.575500
Low-carbon electricity (% electricity)                               1.151000
Primary energy consumption per capita (kWh/person)                   0.000000
Energy intensity level of primary energy (MJ/$2017 PPP GDP)     

In [5]:
data_types = data.dtypes
print(data_types)

Entity                                                               object
Year                                                                  int64
Access to electricity (% of population)                             float64
Access to clean fuels for cooking                                   float64
Renewable-electricity-generating-capacity-per-capita                float64
Financial flows to developing countries (US $)                      float64
Renewable energy share in the total final energy consumption (%)    float64
Electricity from fossil fuels (TWh)                                 float64
Electricity from nuclear (TWh)                                      float64
Electricity from renewables (TWh)                                   float64
Low-carbon electricity (% electricity)                              float64
Primary energy consumption per capita (kWh/person)                  float64
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         float64
Value_co2_em

In [6]:
def remove_functuation(density):
    return density.replace(',','')
data['Entity']=data['Entity'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].apply(remove_functuation)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(float)

In [7]:
#https://stackoverflow.com/questions/18689823/pandas-dataframe-replace-nan-values-with-average-of-columns
#https://www.geeksforgeeks.org/remove-multiple-elements-from-a-list-in-python/
columns=data.columns.tolist()
feature_with_null=[column for column in columns if column not in ['Entity','Year','Primary energy consumption per capita (kWh/person)']]
data.fillna(data[feature_with_null].mean(),inplace=True)
data.dropna(inplace=True)
missing_values = data.isnull().sum()
print(missing_values)

Entity                                                              0
Year                                                                0
Access to electricity (% of population)                             0
Access to clean fuels for cooking                                   0
Renewable-electricity-generating-capacity-per-capita                0
Financial flows to developing countries (US $)                      0
Renewable energy share in the total final energy consumption (%)    0
Electricity from fossil fuels (TWh)                                 0
Electricity from nuclear (TWh)                                      0
Electricity from renewables (TWh)                                   0
Low-carbon electricity (% electricity)                              0
Primary energy consumption per capita (kWh/person)                  0
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         0
Value_co2_emissions_kt_by_country                                   0
Renewables (% equiva

In [8]:
df = data
df.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.613591,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59482,1.64,760.000000,11.986707,3.441610,13283.774348,60.0,652230.0,33.93911,67.709953
1,Afghanistan,2001,4.074574,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89185,1.74,730.000000,11.986707,3.441610,13283.774348,60.0,652230.0,33.93911,67.709953
2,Afghanistan,2002,9.409158,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86215,1.40,1029.999971,11.986707,3.441610,179.426579,60.0,652230.0,33.93911,67.709953
3,Afghanistan,2003,14.738506,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.96822,1.40,1220.000029,11.986707,8.832278,190.683814,60.0,652230.0,33.93911,67.709953
4,Afghanistan,2004,20.064968,10.9,7.75,94224000.0,44.24,0.33,0.0,0.56,...,204.23125,1.20,1029.999971,11.986707,1.414118,211.382074,60.0,652230.0,33.93911,67.709953


In [9]:
def rmOutliers(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# List of columns to check for outliers
columns_to_check = [
    'Access to electricity (% of population)',
    'Access to clean fuels for cooking',
    'Renewable-electricity-generating-capacity-per-capita',
    'Financial flows to developing countries (US $)',
    'Renewable energy share in the total final energy consumption (%)',
    'Electricity from fossil fuels (TWh)',
    'Electricity from nuclear (TWh)',
    'Electricity from renewables (TWh)',
    'Primary energy consumption per capita (kWh/person)',
    'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
    'Value_co2_emissions_kt_by_country',
    'Renewables (% equivalent primary energy)',
    'gdp_growth',
    'gdp_per_capita',
]
print (columns_to_check)

# Apply the function to each group of entities
df_cleaned = df.groupby('Entity').apply(lambda x: rmOutliers(x, columns_to_check)).reset_index(drop=True)

['Access to electricity (% of population)', 'Access to clean fuels for cooking', 'Renewable-electricity-generating-capacity-per-capita', 'Financial flows to developing countries (US $)', 'Renewable energy share in the total final energy consumption (%)', 'Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)', 'Electricity from renewables (TWh)', 'Primary energy consumption per capita (kWh/person)', 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)', 'Value_co2_emissions_kt_by_country', 'Renewables (% equivalent primary energy)', 'gdp_growth', 'gdp_per_capita']


In [10]:
print(f"Total Outliers Removed considering for each country are:{df.count()-df_cleaned.count()}")

Total Outliers Removed considering for each country are:Entity                                                              1167
Year                                                                1167
Access to electricity (% of population)                             1167
Access to clean fuels for cooking                                   1167
Renewable-electricity-generating-capacity-per-capita                1167
Financial flows to developing countries (US $)                      1167
Renewable energy share in the total final energy consumption (%)    1167
Electricity from fossil fuels (TWh)                                 1167
Electricity from nuclear (TWh)                                      1167
Electricity from renewables (TWh)                                   1167
Low-carbon electricity (% electricity)                              1167
Primary energy consumption per capita (kWh/person)                  1167
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         

In [11]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [12]:
scaler= StandardScaler()
columns=data.select_dtypes(include=['float','integer']).columns
data[columns]=scaler.fit_transform(data[columns])

In [13]:
label_encoder=LabelEncoder()
data['Entity']=label_encoder.fit_transform(data['Entity'])

In [14]:
target_variables = ['Value_co2_emissions_kt_by_country', 'Renewable energy share in the total final energy consumption (%)']


In [15]:
# importing all the necessary models that are required to perform the regression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

models={
    'Linear Regression':LinearRegression(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Support Vector Regressor':SVR(),
    'Random Forest Regressor':RandomForestRegressor()
}


In [27]:
data.head()

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,0,-1.658303,-2.557738,-1.496598,-0.493223,-0.483384,0.424678,-0.202319,-0.187525,-0.227228,...,-0.731738,-1.069235,-0.218925,0.0,0.000000,9.608959e-17,-0.239395,0.011997,0.649732,0.797336
1,0,-1.493106,-2.476329,-1.470367,-0.494932,-0.482819,0.445651,-0.202521,-0.187525,-0.225403,...,-0.733627,-1.040080,-0.218967,0.0,0.000000,9.608959e-17,-0.239395,0.011997,0.649732,0.797336
2,0,-1.327910,-2.299862,-1.444137,-0.496783,-0.463218,0.178505,-0.202406,-0.187525,-0.224827,...,-0.734376,-1.139208,-0.218554,0.0,0.000000,-6.922478e-01,-0.239395,0.011997,0.649732,0.797336
3,0,-1.162713,-2.123568,-1.410037,-0.498586,-0.350228,0.138278,-0.201887,-0.187525,-0.224154,...,-0.733826,-1.139208,-0.218292,0.0,0.992157,-6.916532e-01,-0.239395,0.011997,0.649732,0.797336
4,0,-0.997517,-1.947369,-1.373314,-0.500200,0.000000,0.398892,-0.201829,-0.187525,-0.224827,...,-0.734567,-1.197520,-0.218554,0.0,-0.373162,-6.905597e-01,-0.239395,0.011997,0.649732,0.797336


In [30]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

def evaluate_model_with_base_parameters(models_dict):
  for i in range(len(target_variables)):
    print(f"Below are the metrics for the target variable: {target_variables[i]}")
    for name, model in models_dict.items():
      if name == 'Support Vector Regressor':
            # Skipping RFE for SVR as it does not have the feature_importances_ or coef_ attribute
            correlation_matrix =  data.select_dtypes("number").corr()
            target_features = correlation_matrix['Value_co2_emissions_kt_by_country'].drop('Value_co2_emissions_kt_by_country')
            target_features = target_features[abs(target_features)>0.3].index.tolist()
            X = data.drop(columns=target_features)
            y = data[target_variables[i]]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

      else:
            # Applying RFE for other models
            rfe = RFE(estimator=model, n_features_to_select=10)
            X = data.drop(columns=target_variables)
            y = data[target_variables[i]]
            fit = rfe.fit(X, y)
            feature_ranking = pd.DataFrame({'Feature': X.columns, 'Ranking': fit.ranking_}).sort_values(by='Ranking')
            final_features = feature_ranking[feature_ranking['Ranking'] == 1]['Feature'].to_list()
            if 'Entity' not in final_features:
                final_features.append('Entity')
            X = X[final_features]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

      mse = mean_squared_error(y_test, y_pred)
      mae = mean_absolute_error(y_test, y_pred)
      r2 = r2_score(y_test, y_pred)

      print(f"Model: {name}")
      print(f"Mean squared error is: {mse}")
      print(f"Mean absolute error is: {mae}")
      print(f"R2 score error is: {r2}")
      print("\n")


# now calling the function in order to get the outputs
evaluate_model_with_base_parameters(models)


Below are the metrics for the target variable: Value_co2_emissions_kt_by_country
Model: Linear Regression
Mean squared error is: 0.3597169532592369
Mean absolute error is: 0.09811971281293388
R2 score error is: 0.6249215016890043


Model: Decision Tree Regression
Mean squared error is: 0.0031593841465250277
Mean absolute error is: 0.012782480641702459
R2 score error is: 0.996145735619727


Model: Support Vector Regressor
Mean squared error is: 0.9910809960810189
Mean absolute error is: 0.2250028678595537
R2 score error is: 0.12127231628047241


Model: Random Forest Regressor
Mean squared error is: 0.0071118710625579125
Mean absolute error is: 0.01684720190033327
R2 score error is: 0.9929113499969632


Below are the metrics for the target variable: Renewable energy share in the total final energy consumption (%)
Model: Linear Regression
Mean squared error is: 0.23130265388591842
Mean absolute error is: 0.34618676135332666
R2 score error is: 0.78001881367574


Model: Decision Tree Regres